# Regression Used Cars Dataset

https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-data 


#### **1.Dependency**

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from itertools import combinations  # For creating combinations of elements
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.cluster import KMeans

In [2]:
!pip install scikit-learn

#### **2.Utility Function**

In [3]:
def reduce_mem_usage(df, verbose=0):
    """
    Iterate through all numeric columns of a dataframe and modify the data type
    to reduce memory usage.
    """
    # Calculate the initial memory usage of the DataFrame
    start_mem = df.memory_usage().sum() / 1024**2

    # 🔄 Iterate through each column in the DataFrame
    for col in df.columns:
        col_type = df[col].dtype
        # Check if the column's data type is not 'object' (i.e., numeric)
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            # Check if the column's data type is an integer
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                # Check if the column's data type is a float
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)
    # ℹ️ Provide memory optimization information if 'verbose' is True
    if verbose:
        print(f"Memory usage of dataframe is {start_mem:.2f} MB")
        end_mem = df.memory_usage().sum() / 1024**2
        print(f"Memory usage after optimization is: {end_mem:.2f} MB")
        decrease = 100 * (start_mem - end_mem) / start_mem
        print(f"Decreased by {decrease:.2f}%")

    # Return the DataFrame with optimized memory usage
    return df

#### **3.Load Data**

load csv inside data folder

In [4]:
df = pd.read_csv("/kaggle/input/vehicles-csv/vehicles.csv")
df=reduce_mem_usage(df,verbose=True)
df.shape
df[0:4]

Memory usage of dataframe is 84.68 MB
Memory usage after optimization is: 76.54 MB
Decreased by 9.62%


,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN


#### **Split dataset** 

In [5]:
df_train, df_test = train_test_split(df, test_size=0.2)
print(df_train.shape, df_test.shape)

(341504, 26) (85376, 26)


#### **4.Data Preperation**

4.1 Remove first 27 rows and 'country'column ,which are both null records

In [6]:
df = df[27:]
df = df.drop("county", axis=1)
df.reset_index(drop=True, inplace=True)
df[0:4]

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,drive,size,type,paint_color,image_url,description,state,lat,long,posting_date
0,7316814884,https://auburn.craigslist.org/ctd/d/auburn-uni...,auburn,https://auburn.craigslist.org,33590,2014.0,gmc,sierra 1500 crew cab slt,good,8 cylinders,...,NaN,NaN,pickup,white,https://images.craigslist.org/00R0R_lwWjXSEWNa...,Carvana is the safer way to buy a car During t...,al,32.59,-85.480003,2021-05-04T12:31:18-0500
1,7316814758,https://auburn.craigslist.org/ctd/d/auburn-uni...,auburn,https://auburn.craigslist.org,22590,2010.0,chevrolet,silverado 1500,good,8 cylinders,...,NaN,NaN,pickup,blue,https://images.craigslist.org/00R0R_lwWjXSEWNa...,Carvana is the safer way to buy a car During t...,al,32.59,-85.480003,2021-05-04T12:31:08-0500
2,7316814989,https://auburn.craigslist.org/ctd/d/auburn-uni...,auburn,https://auburn.craigslist.org,39590,2020.0,chevrolet,silverado 1500 crew,good,8 cylinders,...,NaN,NaN,pickup,red,https://images.craigslist.org/01212_jjirIWa0y0...,Carvana is the safer way to buy a car During t...,al,32.59,-85.480003,2021-05-04T12:31:25-0500
3,7316743432,https://auburn.craigslist.org/ctd/d/auburn-uni...,auburn,https://auburn.craigslist.org,30990,2017.0,toyota,tundra double cab sr,good,8 cylinders,...,NaN,NaN,pickup,red,https://images.craigslist.org/00x0x_1y9kIOzGCF...,Carvana is the safer way to buy a car During t...,al,32.59,-85.480003,2021-05-04T10:41:31-0500


Create df1, get the average value of deleting NAN and outliers, then assign it to the NAN in df, and then delete the outliers in df to get the latest data set

In [7]:
df1 = df.copy()
odometer_data = df1['odometer']
odometer_data_cleaned = odometer_data.dropna()

kmeans = KMeans(n_clusters=4)
kmeans.fit(odometer_data_cleaned.values.reshape(-1, 1))

outliers = odometer_data_cleaned[kmeans.labels_ == 1]
df1 = df1[~df1['odometer'].isin(outliers)]

average_value = df1['odometer'].mean()
df['odometer'] = df['odometer'].fillna(average_value)

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Use Kmeans to calculate outliers and delete them together with data with a price less than 1,000 ($1,000 is about ￥7,500. Even the price of a second-hand car is too low, so it is also regarded as an outlier and deleted)

In [8]:
price_data = df['price']
kmeans = KMeans(n_clusters=2)
kmeans.fit(price_data.values.reshape(-1, 1))

outliers = price_data[kmeans.labels_ == 1]

df = df[~((price_data.isin(outliers)) | (price_data < 1000))]

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


4.2 Fill missing data

4.2.1. Calculate the missing percentage for each feature

In [9]:
df.isna().sum()/df.shape[0]*100

id               0.000000
url              0.000000
region           0.000000
region_url       0.000000
price            0.000000
year             0.300626
manufacturer     3.978830
model            1.171759
condition       37.689961
cylinders       40.805016
fuel             0.675621
odometer         0.000000
title_status     1.718089
transmission     0.471962
VIN             38.429964
drive           30.287566
size            71.783181
type            21.343673
paint_color     28.602325
image_url        0.010249
description      0.010774
state            0.000000
lat              0.902141
long             0.902141
posting_date     0.010249
dtype: float64

- We can see that "year", "fuel","transmission","posting_date "column has less than 1% of null values. 
- So we can simply drop those rows which have null values in these columns.

4.2.2. Drop  rows which have null values in  "year", "fuel","transmission","posting_date " columns.

In [10]:
df = df.dropna(subset=['year', 'fuel','transmission','posting_date'])

Fill missing values using the modes of "manufacturer", "cylinders", "title_status", "transmission","drive", "type", "paint_color", "lat", "long"

In [11]:
modes = df[["manufacturer", "cylinders", "title_status", "transmission","drive", "type", "paint_color", "lat", "long"]].mode().iloc[0]

df[['manufacturer', 
    'cylinders', 
    'title_status', 
    'transmission',
    'drive', 
    'type', 
    'paint_color', 
    'lat', 
    'long']] = df[['manufacturer', 
                    'cylinders', 
                    'title_status',
                   'transmission',
                    'drive', 
                    'type', 
                    'paint_color', 
                    'lat', 
                    'long']].fillna(value=modes)

4.2.3. for "condition" col:
Fill NAN with randomly selected data from that column to manipulate the more possible results.

In [12]:
condition_options = df["condition"].unique()
df["condition"] = df["condition"].fillna(np.random.choice(condition_options))

checking duplicated records and new_missing percentage

In [13]:
df.duplicated().sum()

0

In [14]:
df.isna().sum()/df.shape[0]*100

id               0.000000
url              0.000000
region           0.000000
region_url       0.000000
price            0.000000
year             0.000000
manufacturer     0.000000
model            1.172602
condition        0.000000
cylinders        0.000000
fuel             0.000000
odometer         0.000000
title_status     0.000000
transmission     0.000000
VIN             38.912334
drive            0.000000
size            71.464554
type             0.000000
paint_color      0.000000
image_url        0.000000
description      0.000533
state            0.000000
lat              0.000000
long             0.000000
posting_date     0.000000
dtype: float64

For all 26 features shown in below:
```
id：id
url:no meaning 
region:region name（no meaning 
region_url:no meaning 
！！price：entry price
！！year：entry year
manufacturer：manufacturer
model：model of vehicle（too much types,"type"is more clear）（no meaning 
！！condition：condition
cylinders：number of cylinders
fuel：fuel type
！！odometer：miles traveled by vehicle
title_status：ownership status（no meaning
transmission:transmission type
VIN：vehicle identification number（no meaning 
drive:qudongleixing
size：（null：72%）（no meaning 
type:car type
paint_color:color of car
image_url:no meaning 
description:listed description of vehicle（no meaning 
county:no meaning 
state：Where to get the license plate number（no meaning 
lat：latitude of listing（no meaning 
long：longitude of listing（no meaning 
！！posting_date:posting_date
```
- After combining the actual meaning of each feature, we can first exclude completely meaningless feature values, which is, (no meaning)

vehicle_age=posting_year - year, get the usage time of this vehicle

In [15]:
df["posting_date"] = pd.to_datetime(df["posting_date"], utc=True)
df["posting_year"] = df["posting_date"].dt.year
df["vehicle_age"] = df["posting_year"] - df["year"]

df[0:4]

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,type,paint_color,image_url,description,state,lat,long,posting_date,posting_year,vehicle_age
0,7316814884,https://auburn.craigslist.org/ctd/d/auburn-uni...,auburn,https://auburn.craigslist.org,33590,2014.0,gmc,sierra 1500 crew cab slt,good,8 cylinders,...,pickup,white,https://images.craigslist.org/00R0R_lwWjXSEWNa...,Carvana is the safer way to buy a car During t...,al,32.59,-85.480003,2021-05-04 17:31:18+00:00,2021,7.0
1,7316814758,https://auburn.craigslist.org/ctd/d/auburn-uni...,auburn,https://auburn.craigslist.org,22590,2010.0,chevrolet,silverado 1500,good,8 cylinders,...,pickup,blue,https://images.craigslist.org/00R0R_lwWjXSEWNa...,Carvana is the safer way to buy a car During t...,al,32.59,-85.480003,2021-05-04 17:31:08+00:00,2021,11.0
2,7316814989,https://auburn.craigslist.org/ctd/d/auburn-uni...,auburn,https://auburn.craigslist.org,39590,2020.0,chevrolet,silverado 1500 crew,good,8 cylinders,...,pickup,red,https://images.craigslist.org/01212_jjirIWa0y0...,Carvana is the safer way to buy a car During t...,al,32.59,-85.480003,2021-05-04 17:31:25+00:00,2021,1.0
3,7316743432,https://auburn.craigslist.org/ctd/d/auburn-uni...,auburn,https://auburn.craigslist.org,30990,2017.0,toyota,tundra double cab sr,good,8 cylinders,...,pickup,red,https://images.craigslist.org/00x0x_1y9kIOzGCF...,Carvana is the safer way to buy a car During t...,al,32.59,-85.480003,2021-05-04 15:41:31+00:00,2021,4.0


Remove meaningless columns

In [16]:
# implement the function
def data_preparation(df:pd.DataFrame)->pd.DataFrame:
    
    y = df['price']
    x =df.drop(columns=['price'])


    return x,y

def feature_engineering(df:pd.DataFrame)->pd.DataFrame:
    
    df.drop(['id','url','region','region_url','model','title_status',
         'VIN','size','image_url','description',
         'state','lat','long','posting_date','posting_year'], 
        axis=1, inplace=True)

    return df

def data_preprocessing(df:pd.DataFrame)->pd.DataFrame:
    x , y =data_preparation(df)
    x=feature_engineering(x)
    return x,y


In [17]:
df_train_x,df_train_y = data_preprocessing(df_train)
df_test_x, df_test_y = data_preprocessing(df_test)

print(df_train_x.shape, df_train_y.shape)

KeyError: "['posting_year'] not found in axis"

#### **Training and Evaluation**